In [ ]:
def scrape_data(url_list):
    all_dfs = []  # List to store DataFrames from all URLs
    
    for url in url_list:
        response = requests.get(url)
        if response.status_code == 200:
            soup = bs(response.content, 'html.parser')

            # Initialize a list to store DataFrames
            dfs = []

            # Find all game divs
            game_divs = soup.find_all('div', class_='vm-stats-game')

            for game_div in game_divs:
                # Find the table within each game div
                table = game_div.find('table', class_='wf-table-inset mod-adv-stats')

                if table:
                    # Extract table data into a DataFrame
                    table_data = []
                    rows = table.find_all('tr')
                    for row in rows:
                        row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
                        table_data.append(row_data)

                    # Convert table_data into a DataFrame and append to dfs list
                    df = pd.DataFrame(table_data[1:], columns=table_data[0])  # Assuming first row is header
                    dfs.append(df)

            # Drop the first DataFrame (if available)
            if dfs:
                dfs.pop(0)  # Remove the first DataFrame
                all_dfs.extend(dfs)
            else:
                print('No DataFrames found for URL:', url)

        else:
            print('Failed to retrieve the webpage. Status code:', response.status_code)

    return all_dfs

def process_data_frames(data_frames):
    processed_dfs = {}  # Dictionary to store processed DataFrames

    for i, df in enumerate(data_frames):
        df_copy = df.copy()

        # Rename the columns
        df_copy.columns = ['name', 'blank', '2K', '3K', '4K', '5K', '1v1', '1v2', '1v3', '1v4', '1v5', 'ECON', 'PL', 'DE']

        # Clean the 'name' column
        df_copy['name'] = df_copy['name'].str.strip().str.replace('\t', '').str.replace('\n', '')

#         # Clean the team names
#         team_names = ['MIBR', 'LEV', 'SEN', 'NRG', 'FUR', '100T', 'LOUD', 'EG', 'G2', 'C9', 'KRÜ']
#         for team in team_names:
#             df_copy['name'] = df_copy['name'].str.replace(team, '').str.strip()

        # Drop all unneeded columns
        df_copy = df_copy.drop(columns=['blank', '2K', '3K', '4K', '5K', 'ECON', 'PL', 'DE'])

        # Extract the first character from each cell in 'col1'
        df_copy['1v1'] = df_copy['1v1'].str.extract(r'^(\d+)')
        df_copy['1v2'] = df_copy['1v2'].str.extract(r'^(\d+)')
        df_copy['1v3'] = df_copy['1v3'].str.extract(r'^(\d+)')
        df_copy['1v4'] = df_copy['1v4'].str.extract(r'^(\d+)')
        df_copy['1v5'] = df_copy['1v5'].str.extract(r'^(\d+)')

        df_copy = df_copy.fillna(0)

        # Convert all columns except the first one to integers
        for column in df_copy.columns[1:]:
            df_copy[column] = df_copy[column].astype(int)

        # Calculate the total clutches and create a new column
        df_copy['clutches'] = df_copy[['1v1', '1v2', '1v3', '1v4', '1v5']].sum(axis=1)

        # Drop all unneeded columns
        df_copy = df_copy.drop(columns=['1v1', '1v2', '1v3', '1v4', '1v5'])

        # Store the processed DataFrame with a unique name
        processed_dfs[f'df_{i+1}'] = df_copy
    
    return processed_dfs